In [1]:
import pandas as pd
import os
# from Bio.Blast.Applications import NcbiblastpCommandline
import glob

Estabelecendo função para leitura de resultados Blast

In [2]:
def read_blast(file):
    return pd.read_csv(file, sep="\t", header=None,
                        names=["query_id", "subject_id", "pct_identity", "aln_length",
                                "n_of_mismatches", "gap_openings", "q_start", "q_end",
                                  "s_start", "s_end", "e_value", "bit_score"])


Leitura tabela de ortologos

In [3]:
outdir = os.path.join('..','data', 'blast_results','All_bacteria','*.tab')  # output
out_files = glob.glob(outdir)
df_ortho = pd.read_table(os.path.join('..','data', 'pairwise_orthologs','All_Bacteria','OrthoFinder','Results_Apr10','Orthogroups', 'Orthogroups.tsv'))
df_ortho = df_ortho.set_index('Orthogroup')
df_ortho = df_ortho.apply(lambda x : x.str.split(','), axis=1)

In [91]:
bacs = df_ortho.columns.tolist()

In [4]:
df_ortho

,Acinetobacter_baumannii,Mycobacterium_leprae,Mycobacterium_tuberculosis,Neisseria_gonorrhoeae,Pseudomonas_aeruginosa,Salmonella_enterica
Orthogroup,,,,,,
OG0000000,NaN,[g1114.t1],"[g1028.t1, g1043.t1, g1053.t1, g1054.t1, g...",NaN,NaN,NaN
OG0000001,NaN,"[g1455.t1, g1572.t1, g304.t1, g39.t1, g393...","[g1027.t1, g1120.t1, g1156.t1, g1185.t1, g...",NaN,NaN,NaN
OG0000002,NaN,NaN,NaN,NaN,"[g1082.t1, g1145.t1, g1155.t1, g1216.t1, g...","[g1564.t1, g246.t1, g2702.t1, g311.t1, g31..."
OG0000003,"[g260.t1, g466.t1]","[g1255.t1, g141.t1, g597.t1]","[g1021.t1, g2834.t1, g3206.t1, g486.t1, g5...",NaN,"[g1172.t1, g1194.t1, g1452.t1, g1825.t1, g...","[g1019.t1, g1149.t1, g1369.t1, g1639.t1, g..."
OG0000004,"[g115.t1, g262.t1, g453.t1]",[g1442.t1],"[g1339.t1, g1474.t1, g1908.t1, g1920.t1, g...",NaN,"[g1362.t1, g1485.t1, g1670.t1, g1853.t1, g...","[g1114.t1, g2030.t1, g2281.t1, g2803.t1]"
...,...,...,...,...,...,...
OG0003215,NaN,NaN,NaN,NaN,NaN,"[g3435.t1, g4081.t1]"
OG0003216,NaN,NaN,NaN,NaN,NaN,"[g3476.t1, g3477.t1]"
OG0003217,NaN,NaN,NaN,NaN,NaN,"[g3588.t1, g3644.t1]"


In [102]:
# for bac in bacs:
#     pd.DataFrame({bac: df_ortho[bac].explode().dropna().unique()}).melt()

Leitura outfiles do BLASTP

In [33]:
df = pd.concat(map(read_blast,out_files)).reset_index().infer_objects()

/tmp/ipykernel_22421/2454776139.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(map(read_blast,out_files)).reset_index().infer_objects()


In [12]:
df_list = []
for bac in df_ortho.columns:
    df_list.append(df_ortho[bac].explode() )

In [18]:
df_ortho2 = pd.concat(df_list, keys=df_ortho.columns.tolist()).swaplevel()

In [23]:
df_ortho2 = df_ortho2.to_frame('Gene')

In [27]:
df_ortho2 = df_ortho2.reset_index().rename(columns = {'level_1':'Bacteria'})

In [34]:
cutoff_ident = 80

Ordena identidade, corta BLASTs com menor identidade menor de 80%, e pega o primeiro match por query_id

In [35]:
df = df\
        .sort_values('pct_identity',ascending=False)\
            .query('pct_identity >= @cutoff_ident')\
                .drop_duplicates('query_id', keep='first')

In [36]:
df

,index,query_id,subject_id,pct_identity,aln_length,n_of_mismatches,gap_openings,q_start,q_end,s_start,s_end,e_value,bit_score
198502,30,g1150.t1,VFG026784(gb|YP_005359856),100.000,508,0,0,51,558,9,516,0.000000e+00,1049.0
262883,0,g1932.t1,VFG015512(gb|NP_252902),100.000,207,0,0,1,207,1,207,4.570000e-154,424.0
107162,673,g4800.t1,VFG044104(gb|NP_253398),100.000,764,0,0,1,764,1,764,0.000000e+00,1568.0
106259,1323,g275.t1,VFG041113(gb|NP_459287),100.000,729,0,0,1,729,1,729,0.000000e+00,1514.0
262570,1,g1986.t1,VFG050298(gb|NP_250643),100.000,226,0,0,1,226,1,226,1.200000e-167,460.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
91158,22,g246.t1,VFG017693(gb|WP_015356917),80.556,324,63,0,1,324,31,354,0.000000e+00,533.0
5959,4400,g2966.t1,VFG021867(gb|WP_010950814),80.460,435,84,1,1,434,1,435,0.000000e+00,545.0
284883,1,g492.t1,VFG049122(gb|WP_012737539),80.440,409,80,0,1,409,1,409,0.000000e+00,704.0
236,236,g279.t1,VFG043447(gb|YP_177831),80.435,138,27,0,1,138,1,138,1.890000e-25,113.0


In [52]:
df_rl = df_ortho2.merge(df, how = 'inner', left_on ='Gene', right_on='query_id')

In [1]:
import re
from Bio import SeqIO
from Bio import Entrez
Entrez.email = input('Email to entrez')


In [63]:
df_rl['gb_accession'] = df_rl['subject_id']\
                        .apply(lambda x: re.search(r'\|(.*?)\)', x).group(1) if re.search(r'\|(.*?)\)', x) else '')

In [65]:
def get_description(id):
    handle = Entrez.efetch(db="protein", id=id, rettype="gb", retmode="text")
    record = SeqIO.read(handle, "genbank")
    handle.close()
    return record.description

In [68]:
df_rl['description'] = df_rl['gb_accession'].apply(get_description)

Exportando para trabalhar no próximo notebook

In [73]:
df_rl.to_csv('../data/processed/relational_df.csv', sep=';', encoding ='utf-8')